In [2]:
from PIL import Image
import soundfile as sf
import numpy as np
import librosa

# Load the image
image_path = 'color.jpeg'
img = Image.open(image_path)

# Parameters
sample_rate = 44100  # Adjust as needed
duration = 30  # Duration of the audio (in seconds)
line_width = 10  # Width of the vertical line

# Get image data as RGB values
image_data = np.array(img)

# Initialize arrays to store pitch and volume data
color_pitch = []
volume = []

# Mapping from color to musical notes (example mapping)
color_to_note = {
    (255, 0, 0): 'C4',  # Red
    (255, 127, 0): 'D4',  # Orange
    (255, 255, 0): 'E4',  # Yellow
    (0, 255, 0): 'F4',  # Green
    (0, 0, 255): 'G4',  # Blue
    (75, 0, 130): 'A4',  # Indigo
    (143, 0, 255): 'B4'   # Violet
}

# Create a list of colors for mapping to notes
colors = list(color_to_note.keys())

# Iterate through vertical lines from left to right
for i in range(image_data.shape[1] // line_width):
    line = image_data[:, i * line_width:(i + 1) * line_width]

    # Find the brightest pixel in the line
    brightest_pixel = np.max(line, axis=(0, 1))

    # Map the brightest color to a musical note or frequency
    closest_color = min(colors, key=lambda color: np.linalg.norm(np.array(brightest_pixel) - np.array(color)))

    # Skip the line if the color is not found
    if closest_color not in color_to_note:
        continue

    # Map the closest color to a musical note or frequency
    note = color_to_note[closest_color]
    frequency = librosa.note_to_hz(note)

    # Use the green channel for volume
    volume_value = brightest_pixel[1] / 255.0
    volume.append(volume_value)

    color_pitch.append(frequency)

# Generate the sound waveform with a smooth transition between consecutive frequencies
t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
sound_wave = np.zeros_like(t)

for i in range(1, len(color_pitch)):
    start_idx = int((i - 1) * len(t) / len(color_pitch))
    end_idx = int(i * len(t) / len(color_pitch))

    # Linearly interpolate between consecutive frequencies and volumes
    freq_interp = np.linspace(color_pitch[i - 1], color_pitch[i], end_idx - start_idx)
    vol_interp = np.linspace(volume[i - 1], volume[i], end_idx - start_idx)

    sound_wave[start_idx:end_idx] = 0.5 * vol_interp * np.sin(2 * np.pi * freq_interp * t[start_idx:end_idx])

# Save the sound to a WAV file
output_path = f'{image_path}_{duration}_.wav'
sf.write(output_path, sound_wave, sample_rate)
print(f"Sound file saved at {output_path}")


Sound file saved at color.jpeg_30_.wav
